## Extract Vitals

## Imports

In [63]:
import sys
sys.path.append("..")

import pandas as pd

import config
from cyclops.querier import SQLQuerier

## Load config, run query

In [64]:
cfg = config.read_config("../configs/default/*.yaml")
querier = SQLQuerier(cfg)
data = querier.run_query()

2022-02-06 17:20:54,039 INFO  cyclops.querier - Loading SQL query...
2022-02-06 17:20:54,043 INFO  cyclops.querier - SQL query file /mnt/nfs/home/krishnanam/vector-delirium/cyclops/queries/gemini/vitals.sql loaded!
2022-02-06 17:20:54,044 INFO  cyclops.querier - Running query on DB...
2022-02-06 17:21:01,517 INFO  cyclops.querier - Extraction from SQL DB completed, in 7.470539808273315 s!
2022-02-06 17:21:01,753 INFO  cyclops.querier - patient_id           737166
genc_id              737166
hospital_id          737166
measurement_name     700354
measure_date_time    700354
measurement_code     700354
measurement_value    700354
dtype: int64 rows extracted!


## Browse through data

In [65]:
pd.set_option('display.max_columns', None)
data

,patient_id,genc_id,hospital_id,measurement_name,measure_date_time,measurement_code,measurement_value
0,,11101327,SMH,None,NaT,None,None
1,,11101479,SMH,None,NaT,None,None
2,,11101623,SMH,None,NaT,None,None
3,,11101955,SMH,None,NaT,None,None
4,,11103215,SMH,None,NaT,None,None
...,...,...,...,...,...,...,...
737161,fff127eb084f3089068163f87575828038137df9ad94d7...,11765421,SMH,None,NaT,None,None
737162,fff54e242506bd7cea6b7ef1f8bbe9cc60ae6b495868e4...,11777483,SMH,None,NaT,None,None
737163,fff70050e86ac22a1ff1236f375ca64af02ec3e216f868...,11262952,SMH,None,NaT,None,None
737164,fff70434cb2b7562e124447611c7fc1114666d40d08b0a...,11715008,SMH,None,NaT,None,None


In [67]:
data['measurement_name'].nunique(), data['measurement_name'].value_counts(), data['genc_id'].nunique(), data['measurement_code'].value_counts()

(15,
 Pulse                           80167
 O2 Saturation (%)               79621
 Diastolic_BP #1                 78242
 Systolic_BP #1                  78228
 Respirations                    77973
 Temperature                     75145
 Oxygen Flow Rate (L/Min)        67530
 Pain Intensity at Rest          54001
 Pain Intensity with Movement    52163
 CAM Results                     32969
 Oxygen Delivery via             13727
 Weight (kg)                      3951
 FiO2 %                           2954
 Systolic_BP #2                   1842
 Diastolic_BP #2                  1841
 Name: measurement_name, dtype: int64,
 39894,
 S_Pulse            80167
 S_O2 Saturation    79621
 Diastolic_S_BP     78242
 Systolic_S_BP      78228
 S_Respirations     77973
 S_Temperature      75145
 S_Oxflowrate       67530
 S_PainIntRest      54001
 S_PainIntMove      52163
 S_CAMResult        32969
 S_OxDeliveryvia    13727
 Wt                  3951
 S_FIO2              2954
 Systolic_S_BP2      1842